In [35]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline
import json
import datetime


from scipy.cluster.hierarchy import dendrogram, linkage

from sklearn.cluster import AgglomerativeClustering
#import scipy.cluster.hierarchy as sch

#import April12.ipynb
#%store -r icd_grouped 
import statsmodels.api as sm
from patsy import dmatrices 

# model selection
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

import seaborn

In [36]:
data= pd.read_csv('group19gender.csv')
data.shape # doesn't have empty rows
#data.head()
#data['SUBJECT_ID'].head(10)


/Users/kathy908000/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(634709, 35)

In [37]:
data.loc[6:11].head(10)

,ROW_ID_x,ICD9_CODE,SHORT_TITLE_x,LONG_TITLE_x,group,group2,lv.1_code,lv.2_code,lv.1_group,lv.2_group,...,bg10,bg11,bg12,bg13,bg14,bg15,bg16,bg17,bg18,bg19
6,252.0,41,Shigella flexneri,Shigella flexneri,0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
7,252.0,41,Shigella flexneri,Shigella flexneri,0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
8,252.0,41,Shigella flexneri,Shigella flexneri,0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
9,255.0,48,Shigella infection NEC,Other specified shigella infections,0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
10,256.0,49,Shigellosis NOS,"Shigellosis, unspecified",0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
11,258.0,51,Botulism food poisoning,Botulism food poisoning,0,1.0,001-009,5.0,INTESTINAL INFECTIOUS DISEASES,Other food poisoning (bacterial),...,0,0,0,0,0,0,0,0,0,0


In [38]:
data = data.drop_duplicates(subset=None, keep='first', inplace=False)
data.shape

(634649, 35)

# Each Row Patient

In [39]:
#data['group2'] = data['group2'].astype(str)
data = data.astype({'group2': str })     # ['group2'].astype(str)
data = data.astype ({'ICD9_CODE': str }) 

In [40]:
data.dtypes

ROW_ID_x         float64
ICD9_CODE         object
SHORT_TITLE_x     object
LONG_TITLE_x      object
group             object
group2            object
lv.1_code         object
lv.2_code        float64
lv.1_group        object
lv.2_group        object
SUBJECT_ID       float64
HADM_ID          float64
GENDER             int64
DOB               object
DOD              float64
DOD_HOSP          object
bg1                int64
bg2                int64
bg3                int64
bg4                int64
bg5                int64
bg6                int64
bg7                int64
bg8                int64
bg9                int64
bg10               int64
bg11               int64
bg12               int64
bg13               int64
bg14               int64
bg15               int64
bg16               int64
bg17               int64
bg18               int64
bg19               int64
dtype: object

In [41]:
#group by groupconcacat
#data = data.groupby('SUBJECT_ID').agg({'group2' : lambda x: ', '.join(x)})
#data = data.groupby('SUBJECT_ID').agg({'group2' : lambda x: ', '.join(x)}, {'ICD9_CODE' :lambda x: ', '.join(x) })

#data = data.groupby('SUBJECT_ID').agg({'group2' : lambda x: ', '.join(x), 'ICD9_CODE' :lambda x: ', '.join(x) })
#data.head()

In [42]:
pdata = data.groupby('SUBJECT_ID')['ICD9_CODE'].apply(lambda group_series: group_series.tolist()).reset_index()
pdata.head(10)


,SUBJECT_ID,ICD9_CODE
0,2.0,"[V053, V290, V3001]"
1,3.0,"[389, 2639, 41071, 4254, 4275, 4280, 5849, 682..."
2,4.0,"[4111, 42, 1363, 2763, 5715, 7994, 7907, E9317..."
3,5.0,"[V053, V290, V3000]"
4,6.0,"[2753, 2767, 2859, 40391, 9972, V1582]"
5,7.0,"[V053, V290, V3001]"
6,8.0,"[7706, 7746, V053, V290, V3001, V502]"
7,9.0,"[4019, 4280, 431, 5070, 5849]"
8,10.0,"[76525, 76515, 7742, V290, V3000]"
9,11.0,[1913]


In [43]:
#to group based on ICD9-code, one hot encoding each category first 2 digits

#pdata['2dgroup'] = list(map(lambda x: x[0:2], list(pdata['ICD9_CODE'])))
#pdata.head()

# find all diagnoses of single patient, make a list, make new dataframe

In [44]:
icd9 = pdata['ICD9_CODE']
icd9

0                                      [V053, V290, V3001]
1        [389, 2639, 41071, 4254, 4275, 4280, 5849, 682...
2        [4111, 42, 1363, 2763, 5715, 7994, 7907, E9317...
3                                      [V053, V290, V3000]
4                   [2753, 2767, 2859, 40391, 9972, V1582]
                               ...                        
46512    [389, 23871, 2449, 2724, 2859, 48241, 4870, 51...
46513    [389, 25000, 27651, 40291, 4280, 5538, 5570, 5...
46514    [2721, 2851, 3004, 4019, 53081, 56881, 5699, 5...
46515    [25000, 2724, 2851, 41401, 4241, 4280, 42833, ...
46516                     [25000, 4019, 4139, 75612, 7861]
Name: ICD9_CODE, Length: 46517, dtype: object

In [48]:
digit = []
row = []
a = []

for i in range(634650):
    for j in icd9.iloc[i]:
        digit = j[0:2]
        row.append(digit)
    a.append(row)

a
    


IndexError: single positional indexer is out-of-bounds

In [ ]:
arr1= []
for i in range(634709):
    if  group2.iloc[i]==1:
        arr1.append(1)
        
    else:
        arr1.append(0)

grouped['bg1'] = arr1

grouped.head()